# **ALGORITMO DE RECOMENDAÇÃO**

**Por filtragem colaborativa baseado em registros dos usuários**

DATASET: https://www.kaggle.com/datasets/jneupane12/movielens

Esse Dataset do [MovieLens](http://movielens.org) contém 20.000.263 avaliações e 465.564 tags em 27.278 filmes. Esses dados foram criados por 138.493 usuários entre 9 de janeiro de 1995 e 31 de março de 2015. Este conjunto de dados foi gerado em 31 de março de 2015 e atualizado em 17 de outubro de 2016

In [1]:
import numpy as np
import pandas as pd
import difflib
from sklearn.metrics.pairwise import cosine_similarity

filmes = pd.read_csv('/kaggle/input/movielens/movies.csv')
notas = pd.read_csv('/kaggle/input/movielens/ratings.csv')


In [2]:
filmes.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
notas.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


**Selecionando apenas as colunas relevantes**

In [4]:
coluna_filmes = filmes[['movieId','title']]
coluna_notas = notas[['userId','movieId','rating']]
filmes_data = pd.merge(coluna_filmes, coluna_notas)
print(filmes_data)

         movieId                   title  userId  rating
0              1        Toy Story (1995)       3     4.0
1              1        Toy Story (1995)       6     5.0
2              1        Toy Story (1995)       8     4.0
3              1        Toy Story (1995)      10     4.0
4              1        Toy Story (1995)      11     4.5
...          ...                     ...     ...     ...
1048570   130219  The Dark Knight (2011)    1339     4.5
1048571   130462          The Boy (2015)    5731     4.0
1048572   130490        Insurgent (2015)    2423     1.0
1048573   130490        Insurgent (2015)    3397     3.5
1048574   130642      Backcountry (2014)    3858     3.0

[1048575 rows x 4 columns]


**Gerando uma tabela pivot:**

 Isso cria uma estrutura onde cada célula corresponde à avaliação de um usuário para um filme. E torna os cálculos de similaridade mais rápidos e diretos.

In [5]:
filmes_pivot = filmes_data.pivot_table(index = ['userId'], columns = ['title'], values = 'rating')
filmes_pivot.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

title,"""Great Performances"" Cats (1998)",$5 a Day (2008),'71 (2014),'Hellboy': The Seeds of Creation (2004),'Neath the Arizona Skies (1934),'R Xmas (2001),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),"'burbs, The (1989)",...,[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),eXistenZ (1999),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nos amours (1983),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Solicitando do usuário o input do filme na qual ele pretende receber recomendações, e através da biblioteca difflib está conseguindo encontrar na lista o filmes com o títulos mais semalhante a entrado do usuário**

In [12]:
filme = input('Escolha seu filme favorito: ')
lista_titulos = coluna_filmes['title'].tolist()
encontrar_filme = difflib.get_close_matches(filme, lista_titulos)
print(encontrar_filme)

Escolha seu filme favorito:  Toy Story


['Toy Story (1995)', 'Toy Story 3 (2010)', 'Toy Story 2 (1999)']


**Caso o filme seja encontrado no dataset ele é selecionado o primeiro da lista, ou seja, o mais semelhante a entrada do usuário, e armazena as notas vinculadas a tal filme**

In [18]:
if encontrar_filme:
    filme = encontrar_filme[0]
    print(f"Filme encontrado: {filme}")
    filme_rating = filmes_pivot[filme]
    print(filme_rating)

else:
    print("Filme não encontrado na base de dados.")

Filme encontrado: Toy Story (1995)
userId
1       NaN
2       NaN
3       4.0
4       NaN
5       NaN
       ... 
7116    4.0
7117    4.0
7118    NaN
7119    5.0
7120    4.5
Name: Toy Story (1995), Length: 7120, dtype: float64


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


**Após encontrar o filme do usuário é feito o cálculo de similaridade e armazenado em um DataFrame, assim encontrando os filmes mais compativéis com a entrada do usuário**

In [17]:
if encontrar_filme:
    filme = encontrar_filme[0]
    print(f"Filme encontrado: {filme}")
    filme_rating = filmes_pivot[filme]

    similaridade = filmes_pivot.corrwith(filme_rating)
    corr_filmes = pd.DataFrame(similaridade, columns=['correlacao'])
    corr_filmes.dropna(inplace=True)
    print(corr_filmes)

else:
    print("Filme não encontrado na base de dados.")

Filme encontrado: Toy Story (1995)


/usr/local/lib/python3.11/dist-packages/numpy/lib/function_base.py:2889: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/usr/local/lib/python3.11/dist-packages/numpy/lib/function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/usr/local/lib/python3.11/dist-packages/numpy/lib/function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
/usr/local/lib/python3.11/dist-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/usr/local/lib/python3.11/dist-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


                                               correlacao
title                                                    
'Round Midnight (1986)                          -1.000000
'Salem's Lot (2004)                             -1.000000
'Til There Was You (1997)                        0.258998
'burbs, The (1989)                               0.054398
'night Mother (1986)                             0.305156
...                                                   ...
eXistenZ (1999)                                  0.196804
loudQUIETloud: A Film About the Pixies (2006)   -0.408248
xXx (2002)                                       0.368637
xXx: State of the Union (2005)                   0.200546
¡Three Amigos! (1986)                            0.127267

[9941 rows x 1 columns]


****

**Já na parte final é armazenado a quantidade de avaliações vinculadas a esse filme.
E por fim é filtrado filmes que possuem quantidade 'X' de avaliações e essa listagem é no final ordenada e apresentada para o usuário**

In [19]:
# Adiciona contagem de avaliações
quantidade_avaliacoes = filmes_data.groupby('title')['rating'].count()
corr_filmes = corr_filmes.join(quantidade_avaliacoes)

# Ordena e exibe recomendações
recomendacoes = corr_filmes[corr_filmes['rating'] > 500].sort_values('correlacao', ascending=False)
print("\nTop 10 recomendações:")
print(recomendacoes.head(10))


Top 10 recomendações:
                                             correlacao  rating
title                                                          
Toy Story (1995)                               1.000000    2569
Toy Story 2 (1999)                             0.766896    1180
Finding Nemo (2003)                            0.557163    1224
Bug's Life, A (1998)                           0.554016    1075
Monsters, Inc. (2001)                          0.534083    1256
Incredibles, The (2004)                        0.476013    1061
Babe (1995)                                    0.430191    1694
Shrek (2001)                                   0.427351    1696
Spider-Man 2 (2004)                            0.427310     858
Wallace & Gromit: The Wrong Trousers (1993)    0.423088     798


In [24]:
filme = input('Escolha seu filme favorito: ')
lista_titulos = coluna_filmes['title'].tolist()
encontrar_filme = difflib.get_close_matches(filme, lista_titulos)

if encontrar_filme:
    filme = encontrar_filme[0]
    print(f"Filme encontrado: {filme}")
    filme_rating = filmes_pivot[filme]

    # Calcula a similaridade
    similaridade = filmes_pivot.corrwith(filme_rating)
    corr_filmes = pd.DataFrame(similaridade, columns=['correlacao'])
    corr_filmes.dropna(inplace=True)

    # Adiciona contagem de avaliações
    quantidade_avaliacoes = filmes_data.groupby('title')['rating'].count()
    corr_filmes = corr_filmes.join(quantidade_avaliacoes)

    # Ordena e exibe recomendações
    recomendacoes = corr_filmes[corr_filmes['rating'] > 500].sort_values('correlacao', ascending=False)
    print("\nTop 10 recomendações:")
    print(recomendacoes.head(10))
else:
    print("Filme não encontrado na base de dados.")

Escolha seu filme favorito:  The Empire Strikes Back


Filme encontrado: Star Wars: Episode V - The Empire Strikes Back (1980)


/usr/local/lib/python3.11/dist-packages/numpy/lib/function_base.py:2889: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/usr/local/lib/python3.11/dist-packages/numpy/lib/function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/usr/local/lib/python3.11/dist-packages/numpy/lib/function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
/usr/local/lib/python3.11/dist-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/usr/local/lib/python3.11/dist-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]



Top 10 recomendações:
                                                    correlacao  rating
title                                                                 
Star Wars: Episode V - The Empire Strikes Back ...    1.000000    2418
Star Wars: Episode IV - A New Hope (1977)             0.759636    2874
Star Wars: Episode VI - Return of the Jedi (1983)     0.719665    2480
Raiders of the Lost Ark (Indiana Jones and the ...    0.488119    2289
Indiana Jones and the Last Crusade (1989)             0.435773    1679
Star Wars: Episode III - Revenge of the Sith (2...    0.400284     652
Toy Story 2 (1999)                                    0.393717    1180
Lord of the Rings: The Two Towers, The (2002)         0.392900    1794
Lord of the Rings: The Return of the King, The ...    0.391421    1619
Lord of the Rings: The Fellowship of the Ring, ...    0.390917    1990
